In [ ]:
! pip install squarify

In [ ]:
# 데이터 읽기
import pandas as pd
import numpy as np

# 데이터 시각화
import plotly.graph_objects as go  # 바차트 시각화
# import plotly.express as px  # 트리맵 시각화 오류남
import squarify

# 자연어 처리
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
data = pd.read_csv('netflix-rotten-tomatoes-metacritic-imdb.csv')

In [ ]:
df = data.copy()

## 기본 데이터 정보확인

- title : 제목
- genre : 장르
- tags : 태그
- languages : 언어
- series or movie : 시리즈 or 영화
- hidden gem score : 영화가 숨겨진 보석인지 확인하는 점수
- country availability : 영화 가용성, 사용가능한 국가
- runtime : 영화 런타임
- director : 감독
- writer : 작가
- actors : 배우들
- view rating : 시청등급
- imdb score : imdb 점수
- rotten tomatoes score : 로튼토마토 점수
- metacritic score : Metacritic 점수
- awards received : 받은 평점
- awards nominated for : 아카데미에서 받은 점수
- boxoffice : 총 매출
- release date : 개봉일
- netflix release date : 넷플릭스 출시일
- production house : 제작사
- netflix link : 넷플릭스 링크
- imdb link : imdb 링크
- imdb votes : imdb 투표수
- image : 이미지링크
- poster : 포스터 링크
- tmdb trailer : tmdb 트레일러 링크
- trailer site : 공식 트레일러 링크

In [ ]:
df.head(1)

In [ ]:
df.info()

In [ ]:
# null값 확인
df.isnull().sum()

## 데이터 전처리

**분석시 불필요한 열 삭제**
- 트레일러 영상이나 이미지, 포스터, 링크 등은 데이터를 분석하는 데 불필요할 것 같아서 삭제했다.
- 날짜와 장르 등을 중점으로 분석하기 위해서 제작사와 매출액들은 제외했다.
- 점수를 통일하기 위해서 metacritic Score를 삭제했다. 

In [ ]:
df = df.drop(columns = [ 'Metacritic Score', 'Boxoffice',
                        'Production House', 'Netflix Link', 'IMDb Link',
                        'Poster', 'TMDb Trailer', 'Trailer Site'], axis = 1)

In [ ]:
df.head(1)

**날짜 데이터 인코딩**

In [ ]:
df['Relese Date'] = pd.to_datetime(df['Release Date'])
df['Netflix Release Date'] = pd.to_datetime(df['Netflix Release Date'])

In [ ]:
df[['Relese Date', 'Netflix Release Date']].head(1)

## 데이터 값 빈도 확인

In [ ]:
colors = ['black',] * 2
colors[0] = 'crimson'

In [ ]:
count = df['Series or Movie'].value_counts()

In [ ]:
count

In [ ]:
# 데이터 시각화 plot.graph_objects 사용
fig = go.Figure(data=[go.Bar(
    x = df["Series or Movie"],
    y = count,
    text = count,
    textposition='auto',
    marker_color=colors 
)])


In [ ]:
# 바차트 제목 및 y축 라벨, x축 라벨 그리기
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'More Movie or Tv Series?',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size= 14),
                  xaxis=dict(
                  title='Category',
                  titlefont_size= 14)
                  )

### movie인 데이터와 series 데이터 분리

In [ ]:
df_movie = df[df['Series or Movie']=='Movie']
df_movie.head(1)

In [ ]:
df_series = df[df["Series or Movie"] == "Series"]
df_series.head(1)

## series 장르 분석 및 시각화

In [ ]:
df_series_gen = df_series.dropna(subset=['Genre'])  # 장르에서 null값 삭제

In [ ]:
colors_10 = ['DarkRed', 'FireBrick','Red', 'Crimson',
             'IndianRed', 'slategray', 'gray', 'dimgrey',
             'DarkSlateGrey', 'black']

In [ ]:
series_gen_list = df_series_gen.Genre.str.split(',')  # 여러 개 장르가 있는 각 데이터를 리스트로 변환

In [ ]:
series_gen_list

In [ ]:
s_gen_list = {}  # 장르 데이터 정리 { 장르명 : 개수}

In [ ]:
for i in series_gen_list:  # 분할된 장르 리스트을 하나씩 불러옴
    for j in i:  # 분할된 장르 리스트에서 값들을 하나씩 불러옴
        if j in s_gen_list:  # 해당 장르가 리스트에 있다면
            s_gen_list[j] += 1  # 장르 count + 1
        else:
            s_gen_list[j] = 1  # 없으면 해당 장르 생성 후 count +1

In [ ]:
# 정리된 장르들을 데이터프레임으로 만듬
s_gen_df = pd.DataFrame(s_gen_list.values(), index= s_gen_list.keys(), columns = {'Counts of Genres in Tv Series'})

In [ ]:
s_gen_df.head(1)

In [ ]:
# 카운트를 기준으로 데이터를 내림차순 정렬
s_gen_df.sort_values(by = 'Counts of Genres in Tv Series',ascending = False,inplace = True)

In [ ]:
s_gen_df.head(1)

In [ ]:
# 상위 10개 장르 추출
top_10_s_gen = s_gen_df[0:10] 

In [ ]:
# 그래프에 상위 10개 장르의 개수들을 넣고 시각화
fig = go.Figure(data=[go.Bar(
    x = top_10_s_gen.index,
    y = top_10_s_gen['Counts of Genres in Tv Series'],
    text = top_10_s_gen['Counts of Genres in Tv Series'],
    textposition='auto',
    marker_color=colors_10
)])

In [ ]:
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most popular genres in Tv Series?',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Genres',
                  titlefont_size=14))

## series 태그 분석 및 시각화

In [ ]:
df_series_tags = df_series.dropna(subset=['Tags'])  # 널값 삭제

In [ ]:
nlp = spacy.load('en')  # 영어에 관한 자연어 처리 사전 로드

In [ ]:
series_tags = df_series_tags['Tags']  # tag값만 가져옴

In [ ]:
series_tags

In [ ]:
doc = []

In [ ]:
for tag in series_tags:  # 문자열 하나로 되어있던것을 토큰 분리
    doc.append(nlp(tag))

In [ ]:
doc = nlp(str(doc))

In [ ]:
words = [token.text  # 리스트화
         for token in doc # 토큰 분리한 데이터를 하나씩 가져옴
         if not token.is_stop and not token.is_punct]  # 문자열이 끝나는점, 구두점 문자가 끝나는 점이 아닐시

In [ ]:
from collections import Counter  # 각 단어당 빈도수 확인
words = Counter(words)

In [ ]:
words

In [ ]:
wordcloud = WordCloud(width = 1400, height = 1400, # 단어별 빈도수로 워드 클라우드 작성 
                background_color ='lavenderblush',  
                min_font_size = 5).generate_from_frequencies(words)  

In [ ]:
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()

## movie 장르 분석 및 시각화
- 분석방법은 위 tv분석방법과 동일

In [ ]:
df_movie_gen = df_movie.dropna(subset=['Genre'])

In [ ]:
movie_gen_list = df_movie_gen.Genre.str.split(',')

In [ ]:
movie_gen_list

In [ ]:
m_gen_list = {}  # 장르 데이터 정리 { 장르명 : 개수}

In [ ]:
for i in movie_gen_list:  # 분할된 장르 리스트을 하나씩 불러옴
    for j in i:  # 분할된 장르 리스트에서 값들을 하나씩 불러옴
        if j in m_gen_list:  # 해당 장르가 리스트에 있다면
            m_gen_list[j] += 1  # 장르 count + 1
        else:
            m_gen_list[j] = 1  # 없으면 해당 장르 생성 후 count +1

In [ ]:
m_gen_list

In [ ]:
m_gen_df = pd.DataFrame(m_gen_list.values(), index= m_gen_list.keys(), columns = {'Counts of Genres in Movie'})

In [ ]:
m_gen_df.sort_values(by = 'Counts of Genres in Movie',ascending = False,inplace = True)

In [ ]:
top_10_m_gen = m_gen_df[0:10] 

In [ ]:
fig = go.Figure(data=[go.Bar(
    x = top_10_m_gen.index,
    y = top_10_m_gen['Counts of Genres in Movie'],
    text = top_10_m_gen['Counts of Genres in Movie'],
    textposition='auto',
    marker_color=colors_10
)])

In [ ]:
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title_text= 'Which are the most popular genres in Movie?',
                  yaxis=dict(
                  title='Quantity',
                  titlefont_size=14),
                  xaxis=dict(
                  title='Genres',
                  titlefont_size=14))

## Movie 태그 분석 및 시각화
- 분석 방법은 위 tv분석방법과 동일

In [ ]:
df_movie_tag = df_movie.dropna(subset=['Tags'])

In [ ]:
movie_tags = df_movie_tag['Tags']

In [ ]:
doc2 = []
for tag in movie_tags:  # 문자열 하나로 되어있던것을 토큰 분리
    doc2.append(nlp(tag))

In [ ]:
doc = nlp(str(doc))
words = [token.text  # 리스트화
         for token in doc 
         if not token.is_stop and not token.is_punct] 

In [ ]:
from collections import Counter  # 각 단어당 빈도수 확인
words = Counter(words)

In [ ]:
wordcloud = WordCloud(width = 1400, height = 1400, # 단어별 빈도수로 워드 클라우드 작성 
                background_color ='lavenderblush',  
                min_font_size = 5).generate_from_frequencies(words) 
plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show() 

## series에서 languages의 빈도수 시각화

In [ ]:
df_series_lan = df_series.dropna(subset=['Languages'])  # languages의 null값 삭제

In [ ]:
series_lan = df_series_lan['Languages']
doc_len = []
for i in series_lan:  # 문자열 하나로 되어있던것을 토큰 분리
    doc_len.append(nlp(i))

In [ ]:
doc_len = nlp(str(doc_len))
len = [i.text  # 리스트화
         for i in doc_len 
         if not i.is_stop and not i.is_punct] 

In [ ]:
from collections import Counter  # 각 단어당 빈도수 확인
len = Counter(len)

In [ ]:
lan = len.copy()

In [ ]:
del(lan[' '])

In [ ]:
lan = pd.DataFrame(dict(lan),index=[0])

In [ ]:
lan =lan.T

In [ ]:
lan.columns = ['count']

In [ ]:
lan.head(5)

In [ ]:
squarify.plot(sizes=lan['count'], label = lan.index, value = lan['count'], )